# Project Title - Prepare data for finding insights in the immigrant's behavior in the U.S in April of 2016
### Data Engineering Capstone Project

#### Project Summary
- This project is about using PySPARK to clean, organize, transform and integrate a series of datasets using a Star Schema approach to enable the insights finding in the data. 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

## Setup

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11").enableHiveSupport().getOrCreate()

## Libraries

In [2]:
# Do all imports and installs here
import pandas as pd
import numpy as np

In [3]:
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *

## Functions

In [4]:
def EDA(df):
    '''
    Description:
    This function get the main information for an EDA
    
    param df: is the dataframe required for the EDA
    '''
    
    # Get number of rows
    rows = df.count()
    # Get summary statistics
    summary_statistics = df.summary().toPandas()
    # Transpose to see variables statistics per row
    summary_statistics = summary_statistics.set_index("summary").T
    # Count number of nulls
    summary_statistics["nulls"] =  rows - summary_statistics["count"].astype("int64")
    # Estimate percentage of nulls
    summary_statistics["% nulls"] = (summary_statistics["nulls"].astype("int64")/rows)*100
    
    return summary_statistics

In [5]:
def groups(df,list_):
    '''
    Description:
    This function creates unique groups of values per specific columns of a dataframe organized in a dictionary
    
    param df: dataframe
    param list: the specific columns to form the groups
    '''        
    
    dict_ = {}
    
    for var in list_:
        value = df.select(var).distinct().agg(collect_list(var)).collect()[0][0]
        dict_[var] = value
    
    return dict_

In [6]:
def intersection_groups(key_a, key_b,dict_a,dict_b):
    '''
    Description:
    This function get the intersection between two groups as a dataframe
    
    param key_a: selected variables of the group a
    param key_b: selected variables of the group b
    param dict_a: dictionary of group a
    param dict_b: dictionary of group b
    '''    
    
    data = []
    
    for a in key_a:
        for b in key_b:
            coincidence = len(list(set(dict_a[a]) & set(dict_b[b]))) 
            if coincidence > 0:
                row = [a,len(dict_a[a]),b,len(dict_b[b]),coincidence]
                data.append(row)
                
    df = pd.DataFrame(data,columns=["var_a","len_list_a","var_b","len_list_b","coincidence"])            
    df = df.sort_values(by="coincidence",ascending = False)

    return df

In [7]:
def transform_rules(df,rules,var):
    '''
    Description:
    This function homologate values between dictionary tables and the fact table
    
    param df: original dataset
    param rules: dataset with the rules and the equivalent key to the df
    param var: the name of the var or key to apply the transformation
    '''    
    df = df.join(rules,df[var] == rules['_c0'],"left")
    new_name = "new_" + str(var)
    df = df.withColumnRenamed("_c1",new_name).drop("_c0")
    
    return df

### Step 1: Scope the Project and Gather Data

#### Scope 
- This project starts with an exploratory analysis of the dataset with the greatest amout of rows, because normally is the fact table, then passing to a series of the analysis process to find keys and respective aggregations and calculation on the rest of the datasets, also normally dimension tables and ends with the integrations of all the datasets in one table for Business Intelligence, using the star schema.

#### Data description and gathering

- **I94 Immigration Data (immigration):** This data comes from the [US National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html). Includes 3096313 rows and 28 columns. 
- **I94 SAS Labels Descriptions**: A data dictionary for the "I94 Inmigration Data".
- **U.S. City Demographics (cities_demographics )**: This data comes from [OpenSoft](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/). Includes 2891 rows and 12 columns.
- **Airport codes (airport_codes):** This is a simple table of airport codes and corresponding cities that comes from [here](https://datahub.io/core/airport-codes#data). Includes 55075 rows and 12 columns.

In [8]:
# Data Gathering
## I94 Immigration Data
path_sas_data = "sas_data" 
## I94 Immigration Data Sample
path_immigration_data = "immigration_data_sample.csv"
## Us cities demographics
path_us_cities_demographics = "us-cities-demographics.csv"
## Airport codes
path_airport_codes = "airport-codes_csv.csv"

In [9]:
## I94 Immigration Data
immigration = spark.read.parquet(path_sas_data)

In [10]:
## Us cities demographics
cities_demographics = (spark.read
                       .option("inferschema","true")
                       .option("header","true")
                       .option("sep",";")
                       .csv(path_us_cities_demographics)
                      )

In [11]:
## Airport codes
airport_codes = (spark
                 .read
                 .option("inferschema","true")
                 .option("header","true")
                 .csv(path_airport_codes)
                )

In [12]:
## Dictionary rules
rules_1 = spark.read.csv("rules_immigrants/1_I94CIT_&_I94RES.txt",sep=";",header=None)
rules_2 = spark.read.csv("rules_immigrants/2_I94PORT.txt",sep=";",header=None)
rules_3 = spark.read.csv("rules_immigrants/3_I94MODE.txt",sep=";",header=None)
rules_4 = spark.read.csv("rules_immigrants/4_I94ADDR.txt",sep=";",header=None)
rules_5 = spark.read.csv("rules_immigrants/5_I94VISA.txt",sep=";",header=None)

### Step 2: Explore and Assess the Data

In [13]:
imminigration_eda = EDA(immigration)
imminigration_eda

summary,count,mean,stddev,min,25%,50%,75%,max,nulls,% nulls
cicid,3096313,3078651.879075533,1763278.0997499449,6.0,1577480.0,3103635.0,4654348.0,6102785.0,0,0.000000
i94yr,3096313,2016.0,0.0,2016.0,2016.0,2016.0,2016.0,2016.0,0,0.000000
i94mon,3096313,4.0,0.0,4.0,4.0,4.0,4.0,4.0,0,0.000000
i94cit,3096313,304.9069344733559,210.02688853063205,101.0,135.0,213.0,512.0,999.0,0,0.000000
i94res,3096313,303.28381949757664,208.58321292789535,101.0,131.0,213.0,504.0,760.0,0,0.000000
i94port,3096313,None,None,5KE,None,None,None,YSL,0,0.000000
arrdate,3096313,20559.84854179794,8.777339475317723,20545.0,20552.0,20560.0,20567.0,20574.0,0,0.000000
i94mode,3096074,1.0736897761487614,0.5158963131657106,1.0,1.0,1.0,1.0,9.0,239,0.007719
i94addr,2943721,51.652482269503544,42.97906231370983,..,10.0,40.0,99.0,ZU,152592,4.928184
depdate,2953856,20573.95283554784,29.35696848166157,15176.0,20561.0,20570.0,20579.0,45427.0,142457,4.600859


**Which variables don't have good amaount of information?**

In [14]:
imminigration_null=imminigration_eda[imminigration_eda["% nulls"]>50].index.tolist()
imminigration_null

['visapost', 'occup', 'entdepu', 'insnum']

**Which variables could have statistical value?**

- 'cicid': NOT, it seems to be just identification but is not clear in the dictionary.
- 'i94yr': NOT, is the year of the data, from 2016 only, so there is not evolution and is from 5 years today.
- 'i94mon': NOT, is the month, but only 4 or April.
- 'i94cit': This is a categorical value but there's a need to transform to something more human-readable.
- 'i94res': This is a categorical value but there's a need to transform to something more human-readable.
- 'i94port': This is a categorical value but there's a need to transform to something more human-readable.
- 'arrdate': This is a date but is not correctly formated.
- 'i94mode': it seems that is model, not mode, and appears to be the way which the immigrant use to arrive at the USA. this could be an interesting variable for analysis but also requires transformation.
- 'i94addr': This is a categorical value but there's a need to transform to something more human-readable.
- 'depdate': This is oppositive of depdate, for only one year, maybe will be interesting to see in average how many time the immigrants states in the U.S.
- 'i94bir': I guess that is the age of the immigrant
- 'i94visa': this has to be key, is another categorical variable that is at 100% and defines which is the type of visa but is represented as a code, so there's a need to transform it to something more readable.
- 'count': NOT, is just a constant.
- 'dtadfile': NOT, It says that CIC does not use.
- 'visapost': NOT, It says that CIC does not use.
- 'occup': NOT, It says that CIC does not use.
- 'entdepa': NOT, It says that CIC does not use.
- 'entdepd': NOT, It says that CIC does not use.
- 'entdepu': NOT, It says that CIC does not use.
- 'matflag': NOT, it seems to be a constant.
- 'biryear': There is a column age, maybe there's a need to verify.
- 'dtaddto': NOT, It says that CIC does not use.
- 'gender': Yes!.
- 'insnum': This is not clear.
- 'airline': This is the airline code, which could be used to determine which is the preferred airline of the immigrants depending on.
- 'admnum': NOT, is an identifier.
- 'fltno': maybe it could be used to complement data about the fly of the immigrant.
- 'visatype': it could be complementary for the i94visa.

**Which could be the possible keys between the dataframes?**

In [15]:
## Immigrants
immigrants_keys = ["i94cit","i94res","i94port","i94addr"]
immigrants_groups = groups(immigration,immigrants_keys)

In [16]:
## Demographics
demographic_keys = ["City","State","State Code"]
demographic_groups = groups(cities_demographics,demographic_keys)

In [17]:
## Airport_code
## remove the US- from 
airport_codes_keys = ["continent","iso_country","iso_region","municipality","gps_code","iata_code","local_code"]
airport_codes_groups = groups(airport_codes,airport_codes_keys)

In [18]:
# Intersections between immigrants and demographics
intersection_set_1 = intersection_groups(immigrants_keys, demographic_keys,immigrants_groups,demographic_groups)
intersection_set_1.head()

,var_a,len_list_a,var_b,len_list_b,coincidence
0,i94addr,457,State Code,49,49


In [19]:
# Interceptions between immigrants and airport codes
intersection_set_2 = intersection_groups(immigrants_keys, airport_codes_keys,immigrants_groups,airport_codes_groups)
intersection_set_2.head()

,var_a,len_list_a,var_b,len_list_b,coincidence
1,i94port,299,iata_code,9042,251
2,i94port,299,local_code,27436,201
4,i94addr,457,iso_country,244,171
3,i94addr,457,continent,7,7
0,i94port,299,gps_code,40850,2


In [20]:
# Interceptios between demographics and airport codes
intersection_set_3 = intersection_groups(airport_codes_keys, demographic_keys,airport_codes_groups,demographic_groups)
intersection_set_3.head()

,var_a,len_list_a,var_b,len_list_b,coincidence
1,municipality,27133,City,567,482
0,iso_country,244,State Code,49,27
2,municipality,27133,State,49,16


The posible joins could be:
 - immigrants left join demographics on 'i94addr' == 'State Code'
 - immigrants left join airport codes is not that clear, maybe there is a need to remove values to enhance the match.
 - demographics join airport_codes 'municipality' == 'City'

In [21]:
## For the second observation:
airport_codes.show()

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

In [22]:
### We can see that the ident, iso_region, the gpc_code and the local_code have some values before the real code of the variable, so I gonna remove this values and try to find I better match to the fact table.
airport_codes = (airport_codes
                 .withColumn("ident",regexp_replace(col("ident"),"0",""))
                 .withColumn("gps_code",regexp_replace(col("gps_code"),"0",""))
                 .withColumn("local_code",regexp_replace(col("local_code"),"0",""))
                 .withColumn("iso_region",split(col("iso_region"), '-')[1])
                )

In [23]:
# We only care for the US data so
airport_codes_us = airport_codes.where(col("iso_country")=="US")

In [24]:
airport_codes_us.show()

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|    A|     heliport|   Total Rf Heliport|          11|       NA|         US|        PA|    Bensalem|       A|     null|         A|-74.9336013793945...|
|   AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|        KS|       Leoti|      AA|     null|        AA|-101.473911, 38.7...|
|   AK|small_airport|        Lowell Field|         450|       NA|         US|        AK|Anchor Point|      AK|     null|        AK|-151.695999146, 5...|
|   AL|small_airport|        Epps Airpark|         820|       NA|         US|     

In [25]:
## Now try again:
airport_codes_keys = ['ident','iso_country','iso_region','gps_code','iata_code']
airport_codes_groups = groups(airport_codes_us,airport_codes_keys)
intersection_set_2 = intersection_groups(immigrants_keys, airport_codes_keys,immigrants_groups,airport_codes_groups)
intersection_set_2

,var_a,len_list_a,var_b,len_list_b,coincidence
2,i94port,299,iata_code,2014,132
3,i94addr,457,ident,20489,97
6,i94addr,457,gps_code,19058,96
5,i94addr,457,iso_region,52,52
0,i94port,299,ident,20489,10
1,i94port,299,gps_code,19058,3
4,i94addr,457,iso_country,1,1


In [26]:
## And try again
intersection_set_3 = intersection_groups(airport_codes_keys, demographic_keys,airport_codes_groups,demographic_groups)
intersection_set_3.head()

,var_a,len_list_a,var_b,len_list_b,coincidence
1,iso_region,52,State Code,49,48
0,ident,20489,State Code,49,44
2,gps_code,19058,State Code,49,44


The posible joins could be:
 - immigrants left join demographics on 'i94addr' == 'State Code'
 - demographics join airport_codes 'iso_region' == 'State Code'

### Which could be the possible aggregations and calculation between the dimendional tables to add value to the fact table?

In [27]:
# For demographics table, the City is the smaller level
cities_demographics.sort("City",ascending=True).show()

+-------+----------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|   City|     State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race| Count|
+-------+----------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|Abilene|     Texas|      31.3|          65212|            60664|          125876|              9367|        8129|                  2.64|        TX|American Indian a...|  1813|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|              9367|        8129|                  2.64|        TX|  Hispanic or Latino| 33222|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|              9367|        8129| 

In [28]:
# The date is repeated by Race, so it appears that there are two tables one for population demographics and the other for races distribution
demographics_popultation = cities_demographics.dropDuplicates(subset=["City"]).drop("City","Race","Count")

In [29]:
# Estimate aggregation per number column grouped by State and State Code
demographics_popultation = (demographics_popultation
                            .groupby(col("State Code").alias("State Code"),col("State").alias("State"))
                            .agg(
                                round(avg('Median Age'),0).alias("Average Median Age")
                                ,sum("Male Population").alias("Male Population")
                                ,sum("Female Population").alias("Female Population")
                                ,sum("Total Population").alias("Total Population")
                                ,sum("Number of Veterans").alias("Number of Veterans")
                                ,sum("Foreign-born").alias("Foreign-born")
                                ,round(avg("Average Household Size"),2).alias("Average Household Size")
                            ))

In [30]:
demographics_popultation.show()

+----------+--------------------+------------------+---------------+-----------------+----------------+------------------+------------+----------------------+
|State Code|               State|Average Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|
+----------+--------------------+------------------+---------------+-----------------+----------------+------------------+------------+----------------------+
|        MT|             Montana|              36.0|          87707|            93587|          181294|             13854|        5977|                  2.28|
|        NC|      North Carolina|              34.0|        1322322|          1448219|         2770541|            133436|      357617|                  2.45|
|        MD|            Maryland|              36.0|         627951|           684178|         1312129|             64143|      229794|                  2.66|
|        CO|            Colorado|             

In [31]:
# Selection of race distribucion data
demographics_race = cities_demographics.select("State Code","State","Race","Count")

In [32]:
# Pivot per race to have one line data per State
demographics_race = (demographics_race
                     .groupby(col("State Code").alias("State Code Race"),col("State").alias("State"))
                     .pivot("Race")
                     .agg(sum("Count")))

In [33]:
demographics_race.show()

+---------------+--------------------+---------------------------------+-------+-------------------------+------------------+--------+
|State Code Race|               State|American Indian and Alaska Native|  Asian|Black or African-American|Hispanic or Latino|   White|
+---------------+--------------------+---------------------------------+-------+-------------------------+------------------+--------+
|             MT|             Montana|                             9684|   4165|                     3349|             10000|  169026|
|             NC|      North Carolina|                            35209| 178740|                  1029446|            354409| 1790136|
|             MD|            Maryland|                            16155| 128839|                   573768|            138644|  594522|
|             CO|            Colorado|                            62613| 148790|                   208043|            703722| 2463916|
|             CT|         Connecticut|                 

In [34]:
# For airport code table the State Code is iso_region so

In [35]:
# In this case the only columns that can generate insighs could be iso_region, type and elevation_ft
airports = (airport_codes_us
            .select("iso_region","type","elevation_ft")
            .groupby(col("iso_region").alias("iso_region"))
            .pivot("type")
            .agg(count("elevation_ft")))
# Fill null values with 0
airports = airports.fillna(0)

In [36]:
airports.show()

+----------+-----------+------+--------+-------------+--------------+-------------+-------------+
|iso_region|balloonport|closed|heliport|large_airport|medium_airport|seaplane_base|small_airport|
+----------+-----------+------+--------+-------------+--------------+-------------+-------------+
|        SC|          0|     6|      36|            4|             9|            0|          160|
|        AZ|          0|    18|     122|            3|            11|            0|          200|
|        LA|          0|    29|     243|            4|            10|            6|          264|
|        MN|          0|    56|      78|            4|             9|           73|          349|
|        NJ|          7|    14|     292|            1|             8|            2|          107|
|        DC|          0|     1|      18|            2|             0|            0|            0|
|        OR|          0|    15|     117|            1|            17|            3|          339|
|        VA|        

#### Clean the Data 

##### Steps: - In the exploratory exercise with clean the dimensional tables and also create new ones, so for the fact table the process is:
- 1. Drop variables with no significant amount of information.
- 2. Drop variables with no statistical value or variables that don't explain or add value.
- 3. Drop null rows for the key column

In [37]:
# Performing cleaning tasks for immigrants

In [38]:
# 1. Null values
immigration = immigration.drop(*imminigration_null) 

In [39]:
# 2. No added value
imminigration_no_value = ['cicid','i94yr','i94mon','count','dtadfile','visapost','occup','entdepa','entdepd','entdepu','matflag','dtaddto','admnum','biryear']
immigration = immigration.drop(*imminigration_no_value)

In [40]:
# 3. Drop null rows for the key column
immigration = immigration.na.drop(subset=["i94addr"])

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

- Because this is a classical business intelligence problem in which we need to answer a series of questions from a set of databases, with no complex relations or grades of normalizations, the Star schema is an excellent choice.

**Dimension Tables:**

- **demographics_popultation:** State Code,State,Average Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size.
- **demographics_race:** State Code Race, State,American Indian and Alaska Native,Asian,Black or African-American,Hispanic or Latino,Whitem.
- **airports:** iso_region,balloonport,closed,heliport,large_airport,medium_airport,seaplane_base,small_airport.
- **rules_1:** c_0, c_1('i94cit','i94res').
- **rules_2:** c_0, c_1('i94port').
- **rules_3:** c_0, c_1('i94mode').
- **rules_4:** c_0, c_1('i94addr').
- **rules_5:** c_0, c_1('i94visa').

**Fact Table:**

- **immigration:** i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,gender,airline,fltno,visatype.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Organize the airport data according with the EDA.
2. Separate the city_demographics table into demographics_popuplation and the demographics_race tables.
3. Clean the immigran table.
4. Add the dictionary rules to the immigran as a fact_table.
5. Combine the demographics_population table with the demographics_race and the airport table as a dimensional_table.
6. Integrate the fact_table with the dimensional_table as immigrants_df.
7. Standarize column names.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [41]:
# 1, 2 y 3 Done in the EDA

In [42]:
fact_table = immigration

In [43]:
# 4. Add the dictionary rules to the immigran table
## Add rules to the immigration table
rules = [rules_1,rules_1,rules_2,rules_3,rules_4,rules_5]
variables = ['i94cit','i94res','i94port','i94mode','i94addr','i94visa']
for i,var in enumerate(variables):
    fact_table = transform_rules(fact_table,rules[i],var)

In [44]:
fact_table.limit(5).toPandas().head()

,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,gender,airline,fltno,visatype,new_i94cit,new_i94res,new_i94port,new_i94mode,new_i94addr,new_i94visa
0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,40.0,1.0,F,QF,00011,B1,"CHINA, PRC",AUSTRALIA,"LOS ANGELES, CA",Air,CALIFORNIA,Business
1,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,32.0,1.0,F,VA,00007,B1,"CHINA, PRC",AUSTRALIA,"LOS ANGELES, CA",Air,NEVADA,Business
2,245.0,438.0,LOS,20574.0,1.0,WA,20582.0,29.0,1.0,M,DL,00040,B1,"CHINA, PRC",AUSTRALIA,"LOS ANGELES, CA",Air,WASHINGTON,Business
3,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,29.0,1.0,F,DL,00040,B1,"CHINA, PRC",AUSTRALIA,"LOS ANGELES, CA",Air,WASHINGTON,Business
4,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,28.0,1.0,M,DL,00040,B1,"CHINA, PRC",AUSTRALIA,"LOS ANGELES, CA",Air,WASHINGTON,Business


In [45]:
# 5. Combine the demographics_population table with the demographics_race and the airport table as a dimensional_table.
dimensional_table = demographics_popultation.join(demographics_race,demographics_popultation["State Code"]==demographics_race["State Code Race"],"left")
dimensional_table =dimensional_table.join(airports,dimensional_table["State Code"]==airports["iso_region"],"left")
dimensional_table = dimensional_table.drop("State Code Race","iso_region")

In [46]:
dimensional_table.limit(5).toPandas().head()

,State Code,State,Average Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State,...,Black or African-American,Hispanic or Latino,White,balloonport,closed,heliport,large_airport,medium_airport,seaplane_base,small_airport
0,MT,Montana,36.0,87707,93587,181294,13854,5977,2.28,Montana,...,3349,10000,169026,0,32,41,1,16,3,238
1,NC,North Carolina,34.0,1322322,1448219,2770541,133436,357617,2.45,North Carolina,...,1029446,354409,1790136,1,11,111,5,15,1,329
2,MD,Maryland,36.0,627951,684178,1312129,64143,229794,2.66,Maryland,...,573768,138644,594522,0,15,80,2,6,1,149
3,CO,Colorado,36.0,1276720,1300079,2576799,162738,271815,2.54,Colorado,...,208043,703722,2463916,1,19,195,1,11,1,277
4,CT,Connecticut,34.0,391277,405821,797098,22076,203538,2.66,Connecticut,...,231822,309992,505674,1,6,96,1,4,3,51


In [49]:
# 6. Integrate the fact_table with the dimensional_table.
immigrants_df = fact_table.join(dimensional_table,fact_table["i94addr"] == dimensional_table["State Code"],"left")
immigrants_df = fact_table.drop("State Code","State")

In [50]:
immigrants_df.limit(5).toPandas().head()

,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,gender,airline,fltno,visatype,new_i94cit,new_i94res,new_i94port,new_i94mode,new_i94addr,new_i94visa
0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,40.0,1.0,F,QF,00011,B1,"CHINA, PRC",AUSTRALIA,"LOS ANGELES, CA",Air,CALIFORNIA,Business
1,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,32.0,1.0,F,VA,00007,B1,"CHINA, PRC",AUSTRALIA,"LOS ANGELES, CA",Air,NEVADA,Business
2,245.0,438.0,LOS,20574.0,1.0,WA,20582.0,29.0,1.0,M,DL,00040,B1,"CHINA, PRC",AUSTRALIA,"LOS ANGELES, CA",Air,WASHINGTON,Business
3,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,29.0,1.0,F,DL,00040,B1,"CHINA, PRC",AUSTRALIA,"LOS ANGELES, CA",Air,WASHINGTON,Business
4,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,28.0,1.0,M,DL,00040,B1,"CHINA, PRC",AUSTRALIA,"LOS ANGELES, CA",Air,WASHINGTON,Business


In [51]:
clean_names = {
    'i94cit':'code_cit',
    'i94res':'code_res',
    'i94port':'code_port',
    'arrdate':'arr_date',
    'i94mode':'code_mode',
    'i94addr':'code_addr',
    'depdate':'dep_date',
    'i94bir':'bir',
    'i94visa':'code_visa',
    'biryear':'bir_year',
    'gender':'gender',
    'airline':'airline',
    'fltno':'flt_no',
    'visatype':'visa_type',
    'new_i94cit':'cit',
    'new_i94res':'res',
    'new_i94port':'port',
    'new_i94mode':'mode',
    'new_i94addr':'addr',
    'new_i94visa':'visa',
    'Average Median Age':'average_median_age',
    'Male Population':'male_population',
    'Female Population':'female_population',
    'Total Population':'total_population',
    'Number of Veterans':'number_veterans',
    'Foreign-born':'foreign_born',
    'Average Household Size':'average_household_size',
    'American Indian and Alaska Native':'american_indian_alaska_native',
    'Asian':'asian',
    'Black or African-American':'black_african_american',
    'Hispanic or Latino':'hispanic_latino',
    'White':'white',
    'iso_region':'region',
    'balloonport':'balloon_port',
    'closed':'closed',
    'heliport':'heliport',
    'large_airport':'large_airport',
    'medium_airport':'medium_airport',
    'seaplane_base':'seaplane_base',
    'small_airport':'small_airport'
}

In [52]:
for var in immigrants_df.columns:
    immigrants_df = immigrants_df.withColumnRenamed(var,clean_names[var])

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

- **Integrity constraints on the relational database (e.g., unique key, data type, etc.)** : in the EDA process the dimensional tables were created by droping duplicates and grouping by keys.

- **Unit tests for the scripts to ensure they are doing the right thing**: Most of the repetitive processes were converted into functions and tested multiple times until a functional result was obtained.

- **Source/Count checks to ensure completeness**: we run the EDA function to verify completeness (No null values per column).

In [53]:
EDA(immigrants_df)

summary,count,mean,stddev,min,25%,50%,75%,max,nulls,% nulls
code_cit,2943721,304.1294307442859,210.93340516818026,101.0,135.0,213.0,512.0,999.0,0,0.000000
code_res,2943721,302.43081834181976,209.53752745145655,101.0,131.0,213.0,509.0,760.0,0,0.000000
code_port,2943721,None,None,5KE,None,None,None,YSL,0,0.000000
arr_date,2943721,20559.843950224902,8.776730783982536,20545.0,20552.0,20560.0,20567.0,20574.0,0,0.000000
code_mode,2943721,1.0541477945769997,0.4522191352762033,1.0,1.0,1.0,1.0,9.0,0,0.000000
code_addr,2943721,51.652482269503544,42.97906231370983,..,10.0,40.0,99.0,ZU,0,0.000000
dep_date,2817737,20574.148197294497,29.624023124568698,15176.0,20561.0,20570.0,20579.0,45427.0,125984,4.279753
bir,2943332,41.7599608199143,17.392602789302135,0.0,30.0,41.0,54.0,114.0,389,0.013215
code_visa,2943721,1.8424704651018218,0.40006697235340466,1.0,2.0,2.0,2.0,3.0,0,0.000000
gender,2551402,None,None,F,None,None,None,X,392319,13.327316


#### 4.3 Data dictionary 

- **code_cit:** code cit.
- **code_res:** code res.
- **code_port:** code port.
- **arr_date*:** arrival date.
- **code_mode:** code mode.
- **code_addr:** code add.
- **dep_date:** departure date.
- **bir:** age.
- **code_visa:** code visa.
- **gender:** gender description.
- **airline:**  airline description.
- **flt_no:** fly number.
- **visa_type:** visa type.
- **cit:** code cit equivalent.
- **res:** code res equivalent.
- **port:** code port equivalent.
- **mode:** fly mode.
- **addr:** code addr equivalent.
- **visa:** visa description.
- **average_median_age:** average median age.
- **male_population:** male population.
- **female_population:** female population.
- **total_population:** total population.
- **number_veterans:** number veterans.
- **foreign_born:** foreign born.
- **average_household_size:** average household size.
- **State:** the state.
- **american_indian_alaska_native:** number of people with race american indian alaska native.
- **asian:** number of people with race asian.
- **black_african_american:** number of people with race african_american.
- **hispanic_latino:** number of people with race white.
- **white:** number of people with race white.
- **balloon_port:** number of airports of type balloon port.
- **closed:** number of airports of type closed.
- **heliport:** number of airports of type heliport.
- **large_airport:** number of airports of type large airport.
- **medium_airport:** number of airports of type medium_airport airport.
- **seaplane_base:** number of airports of type seaplane base airport.
- **small_airport:** number of airports of type small airport airport.

#### Step 5: Complete Project Write Up

1. Clearly state the rationale for the choice of tools and technologies for the project:

- I think that the simplest the better, for that reason I do everything using spark in that way I can avoid increase the complexity of the problem an focus in what is important, the data.

2. Propose how often the data should be updated and why:

- It depends on the slowest time of update of the data sources, but I think that a monthly basis update is ok, to have a control statistics about immigrants to the U.S.

3. Write a description of how you would approach the problem differently under the following scenarios:

- **The data was increased by 100x:** It doesn't matter, spark is design to deal with huges amount of data, so we just need to set a cluster maybe an EMR service on AWS, to scale the capability of spark.

- **The data populates a dashboard that must be updated on a daily basis by 7 am every day:** we have to define a chrono job that can orchestrate the full process, so airflow would be the right option and create a message error system, like python script that checks the number of rows of the table at that hour at that day,that detects when the dag fail and send a email to the technical team to correct the problem.

- **The database needed to be accessed by 100+ people:** In this case, there is a need to use a data warehouse tool, could be Hive or Redshift, to query the data from an interface in a system with the capability to deal with this concurrence.